In [1]:
%%capture
!pip install numpy==1.23.3
!pip install pandas==1.4.3
!pip install Faker==18.6.2
!pip install weasyprint==58.1
!pip install pdfkit==1.0.0

In [2]:
%%capture
import datetime
import pandas as pd
import pdfkit
import numpy as np
import uuid
from faker import Faker
from IPython.display import display, HTML
from weasyprint import HTML, CSS

In [3]:
%%capture
# Initialize Faker object for generating fake data
fake = Faker()

# Set the number of rows to generate
num_rows = 10

# Create a dictionary to hold the data for each column
data = {
    'id': [str(uuid.uuid4()) for _ in range(num_rows)],
    'created': [fake.date_between(start_date='-1y', end_date='today') for _ in range(num_rows)],
    'billing_address': [fake.address() for _ in range(num_rows)],
    'customer_name': [fake.name() for _ in range(num_rows)],
    'line_items': [np.random.choice(['T-shirt', 'Jeans', 'Sweater', 'Dress'], size=np.random.randint(1, 5), replace=False).tolist() for _ in range(num_rows)],
    'special_conditions': [np.random.choice(['true', 'false'], replace=False) for _ in range(num_rows)],
    'completed': [np.random.choice(['true', 'false'], replace=False) for _ in range(num_rows)]
}

# Add the 'due_date' column by iterating over the 'created' column
data['due_date'] = [fake.date_between_dates(date_start, '+7d') for date_start in data['created']]

# Create the DataFrame using the dictionary of data
df = pd.DataFrame(data)

In [4]:
%%capture
# Define the masking colours
def select_col(x):

    light_blue = 'background-color: lightblue'
    light_green = 'background-color: lightgreen'
    red = 'background-color: red'

    df1 = pd.DataFrame('', index=x.index, columns=x.columns)

    special_conditions_mask = x['special_conditions'] == "true"
    completed_true_mask = x['completed'] == "true"
    completed_false_mask = x['completed'] == "false"

    df1.loc[special_conditions_mask, 'special_conditions'] = light_blue
    df1.loc[completed_true_mask, 'completed'] = light_green
    df1.loc[completed_false_mask, 'completed'] = red

    return df1

In [5]:
%%capture
# Style the dataframe
styled_df = df.style.apply(select_col, axis=None).hide(axis='index')

# Convert to a html string
html_string = styled_df.to_html().replace("\\n","<br>")

# Save the HTML string to a file
pdfkit.from_string(html_string, 'output/report.pdf')

True